# Callbacks in Keras
This short tutorial is based on the tutorial found [here](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/). Their code is a lot better documented. If you are new to Neural Networks and do not understand the basics of how they work, have a look at it, since this Tutorial focuses more on `Callbacks`.

In [ ]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
#importing the callback class
from keras.callbacks import Callback
import numpy as np

In [ ]:
# load the dataset
dataset = loadtxt('pima-indians-diabetes.csv', delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]
train_test_split = int(len(X)*.7)
X_train, X_test = X[:train_test_split], X[train_test_split:]
y_train, y_test = y[:train_test_split], y[train_test_split:]

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc']
              )

# Callbacks

The `Callback` class enables us to do operations at certain points of an epoch. These can be

- at the start of each epoch
- at the start of each batch
- at the end of each batch
- at the end of each epoch

and more. But these are probably the most common points in a networks training cycle.

The `Callback` class enables us to look at certain information (metrics) at the current point of the training process and make decisions accordingly. It is for example possible to look at the

- training loss
- training accuracy
- validation loss
- validation accuracy

To see whether or not they are improving. Should they continuously get worse, a dedicated `Callback` could prematurely stop the training process.

Other examplary uses are `Callbacks` that only save the model 

- with the best metrics
- after each epoch and creates a file in which the metrics of each model are listed
- every time the metrics improve

These examples can of course also be combined. 

For this notebook, the second example is created. The other examples can be created accordingly.
Official documentation on Callbacks can be found [here](https://keras.io/callbacks/)

In [ ]:
#creating the class
class ModelSaveCallback(Callback):
    def __init__ (self):
        #inheriting functionalities of the Callback class
        super(Callback, self).__init__()
        
        #creating an attribute for all metrics. Can be used to save metrics from previous
        #epochs for comparison
        self.Metrics = []
    
    #method to be called at the end of an epoch. This is part of the original Callback class
    #that is inherited.
    def on_epoch_end(self, epoch, logs = {}):
        #the argument "logs" contains all the information on the current epoch as a
        #dictionary we ate only extracting the metrics here
        currentMetrics = [logs.get('loss'), logs.get('acc'), logs.get('val_loss'), logs.get('val_acc')] 
        #overwriting the saved metrics
        self.Metrics = currentMetrics
        
        #saving the model
        #The model Attribute is automatically created by the original Callback class, so do
        # not wonder that you did not create it manually yourself.
        # Also the selected model for this attribute is always the one, this Callback class
        # is passed to as an argument, which is done in one of the above cells. 
        self.model.save('models/model_epoch_%s.h5'%(str(epoch)))
                        
        #writing the metrics into a text file
                        
        #clear txt file during the zeroth epoch and write in the current metrics
        if epoch == 0:
            #the 'w' stand for writing. Everything else from the file is deleted beforehand
            with open('model_metrics.txt', 'w') as file:
                file.write('Results at Epoch %s: Loss %s, Acc %s, Val_Loss %s, Val_Acc %s\n\n'
                           %(str(epoch),
                             str(self.Metrics[0]),
                            str(self.Metrics[1]),
                            str(self.Metrics[2]),
                            str(self.Metrics[3])))
        else:
            #the 'a' stands for append. The text is appended to the existing text in the file.
            with open('model_metrics.txt', 'a') as file:
                file.write('Results at Epoch %s: Loss %s, Acc %s, Val_Loss %s, Val_Acc %s\n\n'
                           %(str(epoch),
                             str(self.Metrics[0]),
                            str(self.Metrics[1]),
                            str(self.Metrics[2]),
                            str(self.Metrics[3])))
                

## Fitting the model

Things to do each time you train the network
- delete all the saved models from the `models` directory`, since they will be overwritten


In [ ]:
# fit the keras model on the dataset
#create instance of callback class and add to list of callbacks. Multiple callbacks can
#be attached to a model
saveModelCallback = ModelSaveCallback()
model.fit(X_train,
          y_train,
          epochs = 150, 
          validation_split = .2,
          shuffle = True,
          batch_size = 10,
          callbacks = [saveModelCallback])

In [ ]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Now try changing the class `ModelSaveCallback` so that it only saves a model that fulfills the following criteria:
- loss is lower than that of the last saved model
- validation accuracy is higher than that of the last saved model

Afterwards try playing around with the values you wish to observe and use for your decision-making regarding which model to save.

**Answer below**

The explanatory code from the original class if omitted.

In [ ]:
#creating the class
class ModelSaveCallback_Task(Callback):
    def __init__ (self):
        #inheriting functionalities of the Callback class
        super(Callback, self).__init__()
        
        #creating an attribute for all metrics. Can be used to save metrics from previous
        #epochs for comparison
        #initialize values for accuracy at 0 and for loss at infinity
        self.Metrics = [np.Inf, 0.0, np.Inf, 0.0]
    
    #method to be called at the end of an epoch. This is part of the original Callback class
    #that is inherited.
    def on_epoch_end(self, epoch, logs = {}):

        currentMetrics = [logs.get('loss'), logs.get('acc'), logs.get('val_loss'), logs.get('val_acc')] 
        
        #check if conditions for saving are met
        if float(currentMetrics[0]) < float(self.Metrics[0]) and float(currentMetrics[3]) > float(self.Metrics[3]): 
            self.model.save('models/model_epoch_%s.h5'%(str(epoch)))
            #overwriting the saved metrics only if conditions are met
            self.Metrics = currentMetrics
        else:
            #if the conditions are not met, the rest of the method is skipped
            return
                        
        #since the metrics are initialized with the worst possible values, the first model will always be saved.
        if epoch == 0:
            #the 'w' stand for writing. Everything else from the file is deleted beforehand
            with open('model_metrics.txt', 'w') as file:
                file.write('Results at Epoch %s: Loss %s, Acc %s, Val_Loss %s, Val_Acc %s\n\n'
                           %(str(epoch),
                             str(self.Metrics[0]),
                            str(self.Metrics[1]),
                            str(self.Metrics[2]),
                            str(self.Metrics[3])))
        else:
            #the 'a' stands for append. The text is appended to the existing text in the file.
            with open('model_metrics.txt', 'a') as file:
                file.write('Results at Epoch %s: Loss %s, Acc %s, Val_Loss %s, Val_Acc %s\n\n'
                           %(str(epoch),
                             str(self.Metrics[0]),
                            str(self.Metrics[1]),
                            str(self.Metrics[2]),
                            str(self.Metrics[3])))
                

## Do not forget to clear the `models` directory before running the next cell

In [ ]:
# fit the keras model on the dataset
#create instance of callback class and add to list of callbacks. Multiple callbacks can
#be attached to a model
saveModelCallback = ModelSaveCallback_Task()
model.fit(X_train,
          y_train,
          epochs = 150, 
          validation_split = .2,
          shuffle = True,
          batch_size = 10,
          callbacks = [saveModelCallback])